![alt text](../header.png)

## hls4ml: Bridging Machine Learning and FPGAs for Ultra-Fast Inference  


💡 **High-Level Synthesis for Machine Learning (hls4ml)**  is an open-source library that transforms machine learning models into hardware descriptions optimized for FPGA deployment.

**Key Features of hls4ml:** 

- Converts models from Keras, TensorFlow, PyTorch, and ONNX into High-Level Synthesis (HLS) projects.

- Utilizes tools like Xilinx Vitis HLS and Intel HLS Compiler to generate optimized C++ code for hardware implementation.

- Enhances efficiency by reducing latency and power consumption, making it ideal for AI applications in edge computing.

- Supports quantization and pruning techniques to shrink model size while maintaining accuracy.


For further details:

- GitHub: https://github.com/fastmachinelearning/hls4ml

- Web site: https://fastmachinelearning.org/hls4ml/

---

In [ ]:
import os
import numpy as np
import tensorflow as tf 
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from qkeras import *
from qkeras import QActivation
from qkeras import QDense, QConv2DBatchnorm
import hls4ml
import matplotlib.pyplot as plt


### Path Vitis HLS


As an initial step, the Vivado HLS or Vitis HLS (or another tool) installation directory must be specified.

In [ ]:
os.environ['PATH'] = '/tools/Xilinx/XilinxUnified_2022/Vitis_HLS/2022.2/bin:' + os.environ['PATH']
os.environ['PATH']

#### Load the model (.h5)

In [ ]:

from qkeras.utils import _add_supported_quantized_objects
co = {}
_add_supported_quantized_objects(co)
model = load_model('../models/mnistPQKD.h5', custom_objects=co)


In [ ]:
model.summary()

### High-Level Synthesis for Machine Learning (hls4ml )

Configuration - Granularity: Model

In [ ]:
# granularity='model'

hls_config = hls4ml.utils.config_from_keras_model(model, granularity='model')


# User Configuration 

hls_config['Model']['Precision'] = 'ap_fixed<8, 6>'   
hls_config['Model']['ReuseFactor'] = 16
hls_config['Model']['Strategy'] = 'Resource'

import plotting

print("-----------------------------------")
plotting.print_dict(hls_config)
print("-----------------------------------")




Configuration - Granularity: Name

In [ ]:
# granularity='name'

hls_config = hls4ml.utils.config_from_keras_model(model, granularity='name')

for layer in hls_config['LayerName'].keys():
    # to collect the output from each layer
    # hls_config['LayerName'][layer]['Trace'] = True  
    
    hls_config['LayerName'][layer]['ReuseFactor'] = 16

hls_config['LayerName']['fc1_input_input']['Precision'] = 'ap_fixed<16, 6>'   
hls_config['LayerName']['fc1']['Precision'] = 'ap_fixed<8, 4>'   

hls_config['LayerName']['sigmoid']['Strategy'] = 'Stable'

# To ensure DSPs are optimized, “unrolled” Dense multiplication must be used before synthesizing HLS
hls_config['Model']['Strategy'] = 'Unrolled'


In [ ]:
print("-----------------------------------")
plotting.print_dict(hls_config)
print("-----------------------------------")


### hls4ml with Vitis HLS as backend

In [ ]:
cfg = hls4ml.converters.create_config(backend='vitis')

# cfg['IOType']     = 'io_stream'   # Must set this if using CNNs!
cfg['HLSConfig']  = hls_config      # HLS configuraiton
cfg['KerasModel'] = model           # Keras model to be converted
cfg['OutputDir']  = 'hw/'           # Project name
cfg['Part'] = 'xc7z020clg484-1'     # PYNQ-Z1 or Zedboard: xc7z020clg484-1  ARTIX-7 xc7a35tcsg325-1  # MPSoC xczu4eg-sfvc784-2-e  xczu3eg-sfvc784-1-e

In [ ]:
hls_model = hls4ml.converters.keras_to_hls(cfg)

In [ ]:
hls_model.compile()

#### Hardware synthesis

In [ ]:
hls_model.build(csim=False, export=False)

# hls_model.build(csim=False, export=True, bitfile=True)

In [ ]:

# Vivado version
# hls_config['Flows'] = ['vivado:fifo_depth_optimization']
# hls4ml.model.optimizer.get_optimizer('vivado:fifo_depth_optimization').configure(profiling_fifo_depth=100_000)


# Vitis version
# hls_config['Flows'] = ['vitis:fifo_depth_optimization']
# hls4ml.model.optimizer.get_optimizer('vitis:fifo_depth_optimization').configure(profiling_fifo_depth=100_000)


---
Romina Soledad Molina, Ph.D. - MLab/STI ICTP, Trieste, Italy